In [1]:
import h5py
import xarray as xr
import imageio
import matplotlib.pyplot as plt
import os
import numpy as np
import tidy3d as td
import matplotlib
import matplotlib.patches as patches
import matplotlib.colors as mcolors
from scipy.fft import fftn, ifftn, fftshift
from scipy.interpolate import interp1d
from scipy.signal import argrelextrema
import sys
# Assuming /AutomationModule is in the root directory of your project
sys.path.append(os.path.abspath(rf'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM

tidy3dAPI = os.environ["API_TIDY3D_KEY"]
plt.rc('font', family='Arial')

In [2]:
from matplotlib.colors import LogNorm

def create_movie(sim_result, monitor_lambdas,name='',path="",frames_per_second=1,rem_frames=False, normalize=True, log=False, movie=False):
    # Define the colors: Blue -> White -> Yellow -> Green -> Red
    colors = [
              (1, 1, 1),  # White
              (0, 0, 1),  # Blue
              (1, 1, 0),  # Yellow
              (0, 1, 0),  # Green
              (1, 0, 0), #Red
              (0, 0, 0), # Black
              ]  
    
    # Create a custom colormap
    n_bins = 500  # Number of bins for smooth transition
    cmap = mcolors.LinearSegmentedColormap.from_list("custom_colormap", colors, N=n_bins)
    field_time_out = sim_result["intensity"].squeeze()
    if normalize:
        field_time_out = field_time_out/np.max(field_time_out,axis=(0,1))
    
    # Create a new figure for plotting the shapes
    frames = []
    folder_pics = "pics"
    

    if not os.path.exists(f'{path}/{monitor_lambdas}/{folder_pics}'):
            os.makedirs(f'{path}/{monitor_lambdas}/{folder_pics}',exist_ok=True)
            print(f"Folder {path}/{monitor_lambdas}/{folder_pics} created successfully.")

    for i, time in enumerate(sim_result["t"]):
        if i <50:
            continue
        if os.path.isfile(f'{path}/{monitor_lambdas}/{folder_pics}/frame_{i}.png'):
            frames.append(f'{path}/{monitor_lambdas}/{folder_pics}/frame_{i}.png')
            continue
        value = field_time_out[:,:,i]
        
        fig, ax = plt.subplots(figsize=(7, 9))
        if log:
            value[value==0]=1e-15
            im=ax.imshow(value, 
                     extent =[np.min(sim_result["x"]),np.max(sim_result["x"]), 
                              np.min(sim_result["y"]),np.max(sim_result["y"])], origin ='lower',
                        aspect='auto', cmap=cmap,norm=LogNorm(vmin=np.min(value), vmax=np.max(value)))
        else:
            im=ax.imshow(value, 
                     extent =[np.min(sim_result["x"]),np.max(sim_result["x"]), 
                              np.min(sim_result["y"]),np.max(sim_result["y"])], origin ='lower',
                        aspect='auto', cmap=cmap,vmin=np.min(value), vmax=np.max(value)) 

        
        cbar = fig.colorbar(im, ax=ax, orientation='horizontal', pad=0.03, location="top", shrink=0.3)
        cbar.ax.text(1.05, 0.5, rf"${{|E|^2}}$", va='center', ha='left', transform=cbar.ax.transAxes)
        ax.set_aspect('equal', adjustable='box')
        ax.set_ylabel(rf"x(um)",fontsize=16)
        ax.set_xlabel(rf"y(um)",fontsize=16)
        ax.tick_params(which='major', labelsize=14)
       
        
        # Save the frame
        plt.title(f'Time: {(time*1e12):.4g} ps')
        plt.tight_layout()
        plt.savefig(f'{path}/{monitor_lambdas}/{folder_pics}/frame_{i}.png')
        plt.close(fig)
        frames.append(f'{path}/{monitor_lambdas}/{folder_pics}/frame_{i}.png')
        

        
    if movie:
        name_movie = f'{path}/{name}.mp4' if name else f'{path}/{monitor_lambdas}/Diameter d(t) at output lamb0_{monitor_lambdas}.mp4'
        with imageio.get_writer(name_movie, fps=frames_per_second) as writer:
            for frame in frames:
                image = imageio.imread(frame)
                writer.append_data(image)

    # Optionally, remove the individual frames if no longer needed
    if rem_frames:
        for frame in frames:
            os.remove(frame)
    
    return False


In [3]:
# data = AM.read_hdf5_as_dict("./data/field_data_10x.h5")
# data = AM.read_hdf5_as_dict("./data/field_data_12x_raw_field_Ex_Ey_Ez_lambda0_3p35.h5")
data = AM.read_hdf5_as_dict("./data/field_data_12x_raw_field_Ex_Ey_Ez_lambda0_5p65.h5")
data_2 = AM.read_hdf5_as_dict("./data/field_data_12x_raw_field_Ex_Ey_Ez_lambda0_3p35.h5")
data = {**data, **data_2}

In [4]:
for lambda_0 in data.keys():
    intensity_values = (np.abs(data[lambda_0]["Ex"])**2 + np.abs(data[lambda_0]["Ey"])**2 + np.abs(data[lambda_0]["Ez"])**2).squeeze()
    data[lambda_0]["intensity"] = intensity_values
    create_movie(data[lambda_0],lambda_0, path="./movies/log/12x",frames_per_second=30,movie=True,log=True)

C:\Users\HernandF\AppData\Local\Temp\ipykernel_22372\3552851039.py:19: RuntimeWarning: invalid value encountered in divide
  field_time_out = field_time_out/np.max(field_time_out,axis=(0,1))
C:\Users\HernandF\AppData\Local\Temp\ipykernel_22372\3552851039.py:73: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(frame)
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (700, 900) to (704, 912) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Folder ./movies/log/12x/3.35/pics created successfully.


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (700, 900) to (704, 912) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
